In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import pandas as pd

In [ ]:
from ln_utils import *

In [ ]:
%matplotlib inline

# Parameters

In [ ]:
time_window = 86400*7
#weight = None
weight = "capacity"

# Load temporal data

In [ ]:
graph_files = []

data_dir = "../LNdata/snapshots/"
graph_files +=  [data_dir + f for f in sorted(os.listdir(data_dir)) if ".json" in f][:49] # data files in the second part are corrupted???
MIN_TIME = 1539302399 #Thursday, October 11, 2018 11:59:59 PM
#MAX_TIME = 1553126399 #Wednesday, March 20, 2019 11:59:59 PM

In [ ]:
data_dir = "../LNdata/lncaptures/lngraph/2019/"
graph_files +=  [data_dir + f for f in sorted(os.listdir(data_dir)) if ".json" in f]
MIN_TIME = 1549065601-86400 #Saturday, February 2, 2019 12:00:01 AM
#MAX_TIME = 1552867201 #Monday, March 18, 2019 12:00:01 AM

In [ ]:
data_dir = "../LNdata/"
#graph_files = [data_dir + f for f in sorted(os.listdir(data_dir)) if ".json" in f]
graph_files += [data_dir + f for f in sorted(os.listdir(data_dir)) if ".json" in f][5:]
#MIN_TIME = 1552478399 # Wednesday, March 13, 2019 11:59:59 AM
MAX_TIME = 1553947199 # Saturday, March 30, 2019 11:59:59 AM

graph_files = graph_files[:10]

In [ ]:
nodes, edges = load_temp_data(graph_files[:-1])
print(len(nodes), len(edges))

# I DO NOT UNDERSTAND WHY THERE IS A GAP!!!???

 * there can be bugs in the data? Due to MIN_TIME MAX_TIME filter the edges disappeared from the missing time interval?

edges_lower_cut = edges[(edges["last_update"] > MIN_TIME)]

edges_upper_cut = edges_lower_cut[(edges_lower_cut["last_update"] < MAX_TIME)]

len(edges_lower_cut), len(edges_upper_cut)

edges_upper_cut["last_update"].hist()

In [ ]:
nodes = nodes[(nodes["last_update"] > MIN_TIME) & (nodes["last_update"] < MAX_TIME)]
edges = edges[(edges["last_update"] > MIN_TIME) & (edges["last_update"] < MAX_TIME)]
len(nodes), len(edges)

In [ ]:
edges = edges.sort_values("last_update")

In [ ]:
edges["last_update"].hist(bins=50)

# Analyzing graph snapshots

### Snapshots with the number of edges

In [ ]:
snaphots = get_snapshots(edges, MIN_TIME, MAX_TIME, with_data=True, time_window=time_window)

TMP = nx.Graph()
TMP.add_edges_from(snaphots[0])

%%time
comm = nx.communicability_betweenness_centrality(TMP)

%%time
betw = nx.betweenness_centrality(TMP)

%%time
betw = nx.betweenness_centrality(TMP)

### Calculate centralities for snapshots

# TODO: several betweeness version:
- [current flow](https://networkx.github.io/documentation/networkx-2.2/reference/algorithms/generated/networkx.algorithms.centrality.current_flow_betweenness_centrality.html#networkx.algorithms.centrality.current_flow_betweenness_centrality)
- [communicability](https://networkx.github.io/documentation/networkx-2.2/reference/algorithms/generated/networkx.algorithms.centrality.communicability_betweenness_centrality.html#networkx.algorithms.centrality.communicability_betweenness_centrality)
- [percolation centrality](https://networkx.github.io/documentation/networkx-2.2/reference/algorithms/generated/networkx.algorithms.centrality.percolation_centrality.html#networkx.algorithms.centrality.percolation_centrality)

In [ ]:
stats = get_snapshot_properties(snaphots, weight=weight)

for idx, snap_stat in enumerate(stats):
    snap_stat.to_csv("stat_%i.csv" % idx, index=False)

# Correlation time series

In [ ]:
results = dict([(corr, get_corr_sequence(stats, corr)) for corr in ["pearson","spearman","kendall","w_kendall"]])

### Correlation of centrality measures on adjacent days

In [ ]:
for cent in  ["deg","pr","betw","harm"]:
    show_corr_results(results, cent)